In [3]:
import json
import logging
import math
import os
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from safetensors.torch import load_file
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
MODEL_NAME = "unsloth/llama-2-7b" 
KRON_FACTORS_DIR = "/home/jovyan/shares/SR004.nfs2/kurkin/FisherKronecker/grads_output/Llama-3.1-8B-Instruct_32/fisher_factors_output_1404"
OUTPUT_DIR = f"./{MODEL_NAME.split('/')[-1]}-kron-svd-sensitive"
SENSITIVITY_DICT_PATH = "layers_min_ratio_unsloth_Llama-3.1-8B-Instruct.json"
RANK_ALIGNMENT = 8
MIN_RANK = 1

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        attn_implementation="flash_attention_2",
    )

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

[2025-04-21 14:54:47,399] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/.mlspace/envs/vika_kurkin_clone/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jovyan/.mlspace/envs/vika_kurkin_clone/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [ ]:
model

In [ ]:
model.model.layers[24].mlp

In [14]:
for name, module in model.named_modules():
    print (name, module)

 LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), ep

In [3]:
from compress_llama_with_kronsvd import load_sensitivity_dict, compress_llama_with_kron_svd 
sensitivity_dict = load_sensitivity_dict(SENSITIVITY_DICT_PATH)

# Compress model
compressed_model, compression_stats = compress_llama_with_kron_svd(
    model_name_or_path=MODEL_NAME,
    kron_factors_dir=KRON_FACTORS_DIR,
    sensitivity_dict=sensitivity_dict,
    device="cuda:0",
    rank_alignment=8
)


[2025-04-17 16:53:01,723] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/shares/SR004.nfs2/kurkin/envs/kurkin_312_torch/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jovyan/shares/SR004.nfs2/kurkin/envs/kurkin_312_torch/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Compressing Layers:   0%|          | 0/225 [00:00<?, ?it/s]

Regularizing factors for layer (initial alpha=1.00e-05)...
  Regularizing factor (try 1, alpha=1.00e-05)
  Factor is positive definite (alpha=1.00e-04)
  Regularizing factor (try 1, alpha=1.00e-05)
  Regularizing factor (try 2, alpha=1.00e-04)
  Regularizing factor (try 3, alpha=1.00e-03)
  Factor is positive definite (alpha=1.00e-02)
  Cholesky decomposition successful.
  Performing SVD on transformed matrix (4096x4096)...
  SVD complete. Singular values (top 5): [2.3913357e-05 1.7242742e-06 4.9078540e-07 4.2490385e-07 3.5237093e-07]
  Cholesky factor inverses computed.


...

Regularizing factors for layer (initial alpha=1.00e-05)...
  Regularizing factor (try 1, alpha=1.00e-05)
  Factor is positive definite (alpha=1.00e-04)
  Regularizing factor (try 1, alpha=1.00e-05)
  Factor is positive definite (alpha=1.00e-04)
  Cholesky decomposition successful.
  Performing SVD on transformed matrix (1024x4096)...
  SVD complete. Singular values (top 5): [8.4140293e-06 5.3724966e-07 1.4004478e-07 1.2285818e-07 8.5727088e-08]
  Cholesky factor inverses computed.


Compressing Layers:  97%|█████████▋| 219/225 [2:22:11<01:20, 13.46s/it]

  Factorization complete for layer. New shapes: (328,4096), (1024,328)
Regularizing factors for layer (initial alpha=1.00e-05)...
  Regularizing factor (try 1, alpha=1.00e-05)
  Regularizing factor (try 2, alpha=1.00e-04)
  Regularizing factor (try 3, alpha=1.00e-03)
  Regularizing factor (try 4, alpha=1.00e-02)
  Factor is positive definite (alpha=1.00e-01)
  Regularizing factor (try 1, alpha=1.00e-05)
  Regularizing factor (try 2, alpha=1.00e-04)
  Regularizing factor (try 3, alpha=1.00e-03)
  Regularizing factor (try 4, alpha=1.00e-02)
  Regularizing factor (try 5, alpha=1.00e-01)
  Factor is positive definite (alpha=1.00e+00)
  Cholesky decomposition successful.
  Performing SVD on transformed matrix (4096x4096)...
  SVD complete. Singular values (top 5): [1.1743012e-06 1.0243543e-06 9.8327052e-07 9.5862492e-07 8.6886826e-07]


Compressing Layers: 100%|██████████| 225/225 [2:22:46<00:00, 38.07s/it]

  Cholesky factor inverses computed.
  Factorization complete for layer. New shapes: (824,4096), (4096,824)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.bfloat16)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Token indices sequence length is longer than the specified maximum sequence length for this model (289077 > 131072). Running this sequence through the model will result in indexing errors
Evaluating: 100%|██████████| 36/36 [00:21<00:00,  1.66it/s]
ERROR:root:Failed to evaluate on wikitext2: exp(): argument 'input' (position 1) must be Tensor, not float
Evaluating: 100%|██████████| 12/12 [00:07<00:00,  1.63it/s]
ERROR:root:Failed to evaluate on ptb: exp(): argument 'input' (position 1) must be Tensor, not float
ERROR:root:Failed to evaluate on c4: Unable to find '/home/jovyan/shares/SR004.nfs2/kurkin/FisherKronecker/utils/c4-validation.json'


{'wikitext2': nan, 'ptb': nan, 'c4': nan}

In [19]:

class IndexDataset(Dataset):
    """Simple dataset class for indexed tensors."""
    def __init__(self, tensors):
        self.tensors = tensors

    def __getitem__(self, index):
        return self.tensors[index]

    def __len__(self):
        return len(self.tensors)

def get_test_dataset(name: str, tokenizer, seq_len: int = 2048) -> IndexDataset:
    """Prepare test dataset for perplexity evaluation."""
    def process_data(samples, field_name):
        test_ids = tokenizer("\n\n".join(samples[field_name]), return_tensors='pt').input_ids[0]
        test_ids_batch = []
        nsamples = test_ids.numel() // seq_len

        for i in range(nsamples):
            batch = test_ids[(i * seq_len):((i + 1) * seq_len)]
            test_ids_batch.append(batch)
        test_ids_batch = torch.stack(test_ids_batch)
        return IndexDataset(tensors=test_ids_batch)
    
    if 'wikitext2' in name:
        test_data = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
        return process_data(test_data, 'text')
    if 'ptb' in name:
        test_data = load_dataset('ptb_text_only', 'penn_treebank', split='test')
        return process_data(test_data, 'sentence')
    elif 'c4' in name:
        test_data = load_dataset("json", data_files="utils/c4-validation.json")['train']
        return process_data(test_data[0:2000], 'text')
    else:
        raise ValueError(f"Unknown dataset: {name}")

@torch.no_grad()
def evaluate_perplexity(
    model: nn.Module, 
    dataset: IndexDataset, 
    batch_size: int = 4,
    device: str = "cuda"
) -> float:
    """
    Evaluate model perplexity on a dataset.
    
    Args:
        model: The model to evaluate
        dataset: Dataset of tokenized inputs
        batch_size: Batch size for evaluation
        device: Device for evaluation
        
    Returns:
        Perplexity score (lower is better)
    """
    model.to(device)
    model.eval()
    
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=False
    )
    
    nlls = []
    for batch in tqdm(dataloader, desc="Evaluating"):
        batch = batch.to(device)

        outputs = model(batch, use_cache=False)
        logits = outputs.logits
        
        if not torch.isfinite(logits).all():
            logging.warning("Skipping batch with non-finite logits")
            continue
            
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = batch[:, 1:].contiguous()
        
        loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
        loss = loss_fct(
            shift_logits.reshape(-1, shift_logits.size(-1)), 
            shift_labels.view(-1)
        )
        # Inside the evaluate_perplexity loop, before appending loss
        mean_batch_nll = loss.mean().item()
        print(f"Batch mean NLL: {mean_batch_nll:.4f}")
        nlls.append(loss)

    # Before the return statement
    mean_nll_tensor = torch.cat(nlls, dim=-1).mean()
    print(f"Overall Mean NLL: {mean_nll_tensor.item():.4f}")
    ppl = torch.exp(mean_nll_tensor).item()
    return ppl

def evaluate_model(
    model: nn.Module,
    tokenizer,
    datasets: List[str] = ['wikitext2', 'ptb', 'c4'],
    seq_len: int = 2048,
    batch_size: int = 4,
    device: str = "cuda"
) -> Dict[str, float]:
    """
    Evaluate model on multiple datasets.
    
    Args:
        model: The model to evaluate
        tokenizer: Tokenizer for preprocessing
        datasets: List of dataset names to evaluate on
        seq_len: Maximum sequence length
        batch_size: Batch size for evaluation
        device: Device for evaluation
        
    Returns:
        Dictionary mapping dataset names to perplexity scores
    """
    # Save initial model state
    model.eval()
    use_cache = model.config.use_cache
    model.config.use_cache = False
    model.to(device)
    
    results = {}
    for dataset_name in datasets:
        logging.info(f"Evaluating on {dataset_name}")
        try:
            # Prepare dataset
            test_dataset = get_test_dataset(dataset_name, tokenizer, seq_len)
            
            # Evaluate perplexity
            ppl = evaluate_perplexity(
                model, 
                test_dataset, 
                batch_size=batch_size,
                device=device
            )
            
            results[dataset_name] = ppl
            logging.info(f"{dataset_name} perplexity: {ppl:.4f}")
            
        except Exception as e:
            logging.error(f"Failed to evaluate on {dataset_name}: {e}")
            results[dataset_name] = float('nan')
    
    # Restore model state
    model.config.use_cache = use_cache
    
    return results

In [20]:
dataset = get_test_dataset("wikitext2", tokenizer)
evaluate_perplexity(
    model=compressed_model,
    dataset=dataset,
    batch_size=16)

Evaluating:  11%|█         | 1/9 [00:02<00:18,  2.35s/it]

Batch mean NLL: 13.7500


Evaluating:  22%|██▏       | 2/9 [00:04<00:16,  2.35s/it]

Batch mean NLL: 13.2500


Evaluating:  33%|███▎      | 3/9 [00:07<00:14,  2.35s/it]

...

Evaluating:  89%|████████▉ | 8/9 [00:18<00:02,  2.35s/it]

Batch mean NLL: 13.4375


Evaluating: 100%|██████████| 9/9 [00:20<00:00,  2.31s/it]

Batch mean NLL: 13.5000
Overall Mean NLL: 13.4375


684032.0

In [23]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).eval().to("cuda:1")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
compressed_model.save_pretrained("llama_compressed")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Sequential(
            (0): Linear(in_features=4096, out_features=824, bias=False)
            (1): Linear(in_features=824, out_features=4096, bias=False)
          )
          (k_proj): Sequential(
            (0): Linear(in_features=4096, out_features=328, bias=False)
            (1): Linear(in_features=328, out_features=1024, bias=False)
          )
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Sequential(
            (0): Linear(in_features=4096, out_features=824, bias=False)
            (1): Linear(in_features=824, out_features=4096, bias=False)
          )
        )
        (mlp): LlamaMLP(
          (gate_proj): Sequential(
            (0): Linear(in_features=4096, out_features=1912, bias=False)
            (1): Li